In [1]:
import pandas as pd
from pprint import pprint
import numpy as np
from os import listdir
from os.path import isfile, join
from matplotlib.pyplot import scatter
import matplotlib as plt
import warnings
import math
from pathlib import Path
from os import listdir
from os.path import isfile, join
import pandas as pd
warnings.filterwarnings('ignore')

#### Functions From Current Pose Script

In [2]:
# process csv

def mean_ten_still_frames(pose_csv):
    pose_df = pd.read_csv(pose_csv)
    pose_diff = pose_df.diff()
    rows_total_diff = pose_diff.sum(axis=1)
    rows_total_diff = [abs(i) for i in rows_total_diff]
    ten_rows_diff = []
    for i in range(len(rows_total_diff)-10):
        ten_rows_diff.append((i, sum(rows_total_diff[i:(i+10)])))
    best_ten = sorted(ten_rows_diff, key=lambda x: x[1], reverse=False)
    still_point = best_ten[0][0]
    stillest_ten = pose_df.iloc[still_point:still_point+10, :]
    mean = np.mean(stillest_ten, axis=0)
    return mean[1:]

def x_y_points(data):
    """
    from array of average of points in a single 'pose_keypoints_2d' array
    finds x and y corridnates and return two lists
    """
    x_warrior = []
    y_warrior = []
    c_warrior = [] # certainity of pose
    for n in range(len(data)):
        if (n%3) == 0:
            x_warrior.append(data[n])
        elif (n%3) == 1:
            y_warrior.append(data[n])
        elif (n%3) == 2:
            c_warrior.append(data[n])

    return x_warrior, y_warrior


# In progress

def straight_arms_slope(x, y, min_slope=-0.07, max_slope=0.0481):
    """
    input array of 25 x corridnates and array of 25 y corridinates from openpose (x_y_points(data))
    output is slope of the line from one hand to another
    perfectly straight arms would have a slope of zero.
    7:"LWrist" and 4:"RWrist"
    0 - straight
    1 - not straight
    returns slope and label
    """
    slope = (y[7]-y[4])/(x[7]-x[4])
    if min_slope <= slope <= max_slope:
        return slope, 0.0
    else:
        return slope, 1.0

def straight_arms_area(x, y, max_area = 40, max_slope = 0.07):
    """
    7:"LWrist"
    5: 'LShoulder'
    4:"RWrist"
    2: 'RShoulder'
    1: 'Neck'
    """
    d1 = (x[2]-x[5], y[2]-y[5])
    d2 = (x[4]-x[7], y[4]-y[7])
    A = .5 *abs((d1[0]*d1[1])-(d2[0]*d2[1]))
    arms_len = np.sqrt((x[7]-x[0])**2+(y[7]-y[0])**2) # 7, 0
    slope_shoulder = (y[5]-y[2])/(x[5]-x[2])

    if abs(A/arms_len) <= max_area and slope_shoulder <= max_slope:
        return (A/arms_len, slope_shoulder), 0.0
    else:
        return (A/arms_len, slope_shoulder), 1.0 # 5 wrong

def straight_arms(x, y, min_slope=-0.25, max_slope=0.25):
    slope_shoulder = (y[5]-y[2])/(x[5]-x[2])
    if min_slope <= slope_shoulder <= max_slope:
        return straight_arms_slope(x, y)
    else:
        return straight_arms_area(x, y)


def shoulders_up(x, y, max_angle=10): # 0.3
    """
    1:"Neck",
    2:"RShoulder",
    5:"LShoulder".
    looks at line from left shoulder to neck, and line from right shoulder to neck
    if either are not straight returns 1
    if both are flat (slope of 0 or close to 0) returns 1
    """
    left_degrees = math.degrees(math.atan2(y[5]-y[1], x[5]-x[1]))
    right_degrees = math.degrees(math.atan2(y[1]-y[2], x[1]-x[2]))
#     left_degrees = math.degrees(math.atan((y[5]-y[1])/(x[5]-x[1])))
#     right_degrees = math.degrees(math.atan((y[1]-y[2])/(x[1]-x[2])))  # no difference
    slope_shoulder = (y[5]-y[2])/(x[5]-x[2])
    if (left_degrees <= max_angle and right_degrees <= max_angle) and slope_shoulder <= 0.25:
        return left_degrees, right_degrees, 0.0
    else:
        return left_degrees, right_degrees, 1.0

def hips_square(x, y, max_slope=0.1):
    """
    9:"RHip" and 12:"LHip"
    straight line (square hips) would have a slope of 0
    0 - stright
    1 - not straight
    """
    slope = (y[9] - y[12])/(x[9]-x[12])
    if -max_slope <= slope <= max_slope:
        return slope, 0.0
    else:
        return slope, 1.0

# Working well

def straight_torso(x, y, min_slope=9):
    """
    1:"Neck" and 8:"MidHip"
    perfect would be a vertial line, so steep/high slope is ideal
    0 - straight
    1 - not straight
    returns slope and label
    """
    slope = (y[1] - y[8])/(x[1]-x[8])
    if abs(slope) >= min_slope:
        return slope, 0.0
    else:
        return slope, 1.0


def torso_forward(x, y, min_slope=-0.2):
    """
    1:"Neck" and 8:"MidHip"
    perfect would be a vertial line, so steep/high slope is ideal
    for too far forward we see if the slope if larger than the min slope
    0 - not too far forward
    1 - too far forward
    returns slope and label
    """
    rev_slope = (x[1]-x[8])/(y[1] - y[8])
    if rev_slope <= min_slope:
        return rev_slope, 1.0
    else:
        return rev_slope, 0.0


def torso_backward(x, y, min_slope=0.02):
    """
    1:"Neck" and 8:"MidHip"
    perfect would be a vertial line, so steep/high slope is ideal
    swtiches x and y for easier computation, want reversed slope to be zero if straight
    for too far forward we see if the slope if larger than the min slope
    0 - not too far forward
    1 - too far forward
    returns slope and label
    """
    rev_slope = (x[1]-x[8])/(y[1] - y[8])
    if rev_slope >= min_slope:
        return rev_slope, 1.0
    else:
        return rev_slope, 0.0

def head_front(x, y, max_ratio_diff=0.5, side='right'):
    """
    0:"Nose"
    15:"REye"
    16:"LEye"
    17:"REar"
    18:"LEar"
    Compares distance from left eye to right eye
    If looking forward eye to eye distance will be larger and closer to ear to ear distance
    If looking if head is front they will be small, and much smaller than ear to ear distance
    Divide by length from ear to ear to normalize and account for different distance
    label 0 - head is front
    label 1 - head is not facing the front (facing the side)
    """
    if side == 'right':
        ear_dist = np.sqrt((x[17]-x[0])**2+(y[17]-y[0])**2)
        eye_dist = np.sqrt((x[15]-x[0])**2+(y[15]-y[0])**2)
    else:
        ear_dist = np.sqrt((x[18]-x[0])**2+(y[18]-y[0])**2)
        eye_dist = np.sqrt((x[16]-x[0])**2+(y[16]-y[0])**2)
    ratio = eye_dist/ear_dist
    if ratio > max_ratio_diff:
        return ratio, 1.0
    else:
        return ratio, 0.0


def front_knee_obtuse(x, y, max_angle=75, side='right'):
    """
    10:"RKnee",
    11:"RAnkle",
    13:"LKnee",
    14:"LAnkle"
    """
    if side =='right':
        degrees = math.degrees(math.atan2(y[14]-y[13], x[14]-x[13]))
    else:
        degrees = math.degrees(math.atan2(y[11]-y[10], x[11]-x[10]))
    if degrees < max_angle:
        return degrees, 1.0
    else:
        return degrees, 0.0


def front_knee_acute(x, y, min_angle=100, side='right'):
    """
    10:"RKnee",
    11:"RAnkle",
    13:"LKnee",
    14:"LAnkle"
    """
    if side =='right':
        degrees = math.degrees(math.atan2(y[14]-y[13], x[14]-x[13]))
    else:
        degrees = math.degrees(math.atan2(y[11]-y[10], x[11]-x[10]))
    if degrees > min_angle:
        return degrees, 1.0
    else:
        return degrees, 0.0


def step_too_narrow(x, y, min_ratio=0.61):
    """
    4:"RWrist",
    7:"LWrist",
    11:"RAnkle",
    14:"LAnkle".
    compares arm span to distance between feet
    if feet are wide enough, the distance between feet will be similar
    to the distance between arms
    label - 0 feet are wide enough
    label - 1 feet are too narrow
    """
    arm_distance = np.sqrt((x[7]-x[4])**2+(y[7]-y[4])**2)
    feet_disatance = np.sqrt((x[11]-x[14])**2+(y[11]-y[14])**2)
    ratio = feet_disatance/arm_distance
    if ratio < min_ratio:
        return ratio, 1.0
    else:
        return ratio, 0.0


def step_too_wide(x, y, max_ratio=0.9):
    """
    4:"RWrist",
    7:"LWrist",
    11:"RAnkle",
    14:"LAnkle".
    compares arm span to distance between feet
    if feet are wide enough, the distance between feet will be similar
    to the distance between arms
    label - 0 feet are wide enough
    label - 1 feet are too narrow
    """
    arm_distance = np.sqrt((x[7]-x[4])**2+(y[7]-y[4])**2)
    feet_disatance = np.sqrt((x[11]-x[14])**2+(y[11]-y[14])**2)
    ratio = feet_disatance/arm_distance
    if ratio > max_ratio:
        return ratio, 1.0
    else:
        return ratio, 0.0

def warroir2_label_csv(pose_csv, side='right'):
    """
    takes averages of all rows (2d_points)
    OLD order: head_front, sholders, arms, torso forward, torso backward hips, knee acute, knee obtuse, step wider
    1 - needs to be adjusted
    0 - good
    Order for 9 digit labeling:
    1. arms
    2. front_knee_obtuse
    3. front_knee_acute
    4. head_sideways
    5. hips_angled
    6. narrow_step
    7. shoulders_up
    8. torso_forward
    9. torso_backward
    10. wide_step
    """

    x, y = x_y_points(np.array(mean_ten_still_frames(pose_csv))) # average for each point accros all ten frames
    labels = []
    values = []
    # 1 arms
    slope, label = straight_arms(x, y)
    labels.append(label)
    values.append(slope)
    # 2 and 3 front_knee_obtuse and front_knee_acute
    if side == 'right':
        obtuse_angle, obtuse_label = front_knee_obtuse(x, y, side='right')
        acute_angle, acute_label = front_knee_acute(x, y, side='right')
    else:
        obtuse_angle, obtuse_label = front_knee_obtuse(x, y, side='left')
        acute_angle, acute_label = front_knee_acute(x, y, side='left')
    labels.append(obtuse_label)
    values.append(obtuse_angle)
    labels.append(acute_label)
    values.append(acute_angle)
    # 4 head_sideways
    ratio, label = head_front(x, y)
    labels.append(label)
    values.append(ratio)
    # 5 hips_angled
    slope, label = hips_square(x, y)
    labels.append(label)
    values.append(slope)
    # 6 narrow_step
    ratio, label = step_too_narrow(x, y)
    labels.append(label)
    values.append(ratio)
    # 7 shoulders_up
    left_slope, right_slope, label = shoulders_up(x, y)
    labels.append(label)
    values.append((left_slope, right_slope))
    # 8 torso_forward
    slope, label = torso_forward(x, y)
    labels.append(label)
    values.append(slope)
    # 9 torso_backward
    slope, label = torso_backward(x, y)
    labels.append(label)
    values.append(slope)
    # 10 too wide step
    ratio, label = step_too_wide(x, y)
    labels.append(label)
    values.append(ratio)
    return labels, values


In [3]:
## Checking Functions

def plot(csv):
    x, y = x_y_points(np.array(mean_ten_still_frames(csv)))
    scatter(x, y)

### Create DF of Correct Labels 

In [4]:
new_labels_df = pd.read_csv('labels-new-421.csv')

new_labels_df = new_labels_df.iloc[:, :-1]
new_labels_df['name'] = [s[0:-4] for s in new_labels_df['video_name (Naming: warrior2_0000000000_katja1.avi)']]

means_df = pd.DataFrame()

for name in new_labels_df['name'].unique():
    if name != "warrior2_correct_1":
        mean = mean_ten_still_frames("train_data_csv/" + name + ".csv")
        means_df[name] = mean
        
meansT = means_df.T.reset_index()
meansT = meansT.rename(columns={'index': 'name'})

tags_dict = {'arms':0, 
        'front_knee_obtuse': 1,
        'front_knee_acute':2, 
        'head_sideways': 3,
        'hips_angled':4,
        'narrow_step':5,
        'shoulders_up':6,
        'torso_forward':7,
        'torso_backward':8,
            'wide_step':9}

def string_to_tensor(y, tags_dict):
    """This function creates tensors as np arrays
    y - is tags column of the datafram and path is path to dict"""

    try:
        tags = y.split(' ')
    except:
        return np.zeros(10)   

    tags_tensor = np.zeros(10)

    for label in tags:
        if label in tags_dict.keys():
            tags_tensor[tags_dict[label]] = 1
    
    return tags_tensor

new_labels_df["y_true"] = new_labels_df["labels"].apply(lambda x: string_to_tensor(x, tags_dict))

means_poses_labels = pd.merge(meansT, new_labels_df[['name', 'y_true']])


In [5]:
means_poses_labels.head(1)

,name,0,1,2,3,4,5,6,7,8,...,66,67,68,69,70,71,72,73,74,y_true
0,warrior2_knee1_1,704.2094,131.7818,0.873184,652.1342,196.4106,0.881475,591.7704,194.3839,0.768179,...,417.8212,608.2607,0.775085,409.9247,602.012,0.7911,430.1526,586.2949,0.664928,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### Add Predicted Labels 

In [6]:
rules_1 = pd.DataFrame()
rules_1 = means_poses_labels

predicted_labels = []
different_than_true = []
values_from_functions = []

for pose in rules_1['name']:
    label, values = warroir2_label_csv("train_data_csv/" + pose + ".csv")
    predicted_labels.append(label)
    values_from_functions.append(values)

rules_1['predicted_label'] = predicted_labels
rules_1['same'] =[np.array_equal(np.array(list(rules_1[rules_1['name']==name]['y_true'])),
                                 np.array(list(rules_1[rules_1['name']==name]['predicted_label']))) for name in rules_1['name']]

# Add coloumns for each pose's label
true = rules_1['y_true']

for key in tags_dict.keys():
    rules_1[key+"_true"] = [t[tags_dict[key]] for t in true]
    rules_1[key +"_predicted"] = [t[tags_dict[key]] for t in predicted_labels]

In [7]:
rules_1.head(1)

,name,0,1,2,3,4,5,6,7,8,...,narrow_step_true,narrow_step_predicted,shoulders_up_true,shoulders_up_predicted,torso_forward_true,torso_forward_predicted,torso_backward_true,torso_backward_predicted,wide_step_true,wide_step_predicted
0,warrior2_knee1_1,704.2094,131.7818,0.873184,652.1342,196.4106,0.881475,591.7704,194.3839,0.768179,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
len(rules_1[rules_1.same==True]), len(rules_1[rules_1.same==False])

(12, 18)

In [9]:
# Individual poses df's

arms_df = rules_1[['name', 'arms_true', 'arms_predicted']]
arms_df['value'] = [x[0] for x in values_from_functions]

knee_obtuse_df = rules_1[['name', 'front_knee_obtuse_true', 'front_knee_obtuse_predicted']]
knee_obtuse_df['value'] = [x[1] for x in values_from_functions]

knee_acute_df = rules_1[['name', 'front_knee_acute_true', 'front_knee_acute_predicted']]
knee_acute_df['value'] = [x[2] for x in values_from_functions]

head_df = rules_1[['name', 'head_sideways_true', 'head_sideways_predicted']]
head_df['value'] = [x[3] for x in values_from_functions]

hips_df = rules_1[['name', 'hips_angled_true', 'hips_angled_predicted']]
hips_df['value'] = [x[4] for x in values_from_functions]

narrow_step_df = rules_1[['name', 'narrow_step_true', 'narrow_step_predicted']]
narrow_step_df['value'] = [x[5] for x in values_from_functions]

shoulders_up_df = rules_1[['name', 'shoulders_up_true', 'shoulders_up_predicted']]
shoulders_up_df['value'] = [x[6] for x in values_from_functions]

torso_forward_df = rules_1[['name', 'torso_forward_true', 'torso_forward_predicted']]
torso_forward_df['value'] = [x[7] for x in values_from_functions]

torso_backward_df = rules_1[['name', 'torso_backward_true', 'torso_backward_predicted']]
torso_backward_df['value'] = [x[8] for x in values_from_functions]

wide_step_df = rules_1[['name', 'wide_step_true', 'wide_step_predicted']]
wide_step_df['value'] = [x[9] for x in values_from_functions]


In [10]:
for i, j in zip(rules_1.iloc[0, :]['y_true'], rules_1.iloc[0, :]['predicted_label']):
    print("true :", i, " - predicted : ", j)

true : 0.0  - predicted :  0.0
true : 0.0  - predicted :  0.0
true : 1.0  - predicted :  1.0
true : 0.0  - predicted :  0.0
true : 0.0  - predicted :  0.0
true : 0.0  - predicted :  0.0
true : 0.0  - predicted :  0.0
true : 0.0  - predicted :  0.0
true : 0.0  - predicted :  0.0
true : 0.0  - predicted :  0.0


In [11]:
rules_1.iloc[0, :]['y_true']

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [12]:
len(rules_1.iloc[0, :]['predicted_label'])

10

In [13]:
def total_accuracy(row):
    correct = 0
    true = row['y_true']
    predicted = row['predicted_label']
    for i, j in zip(true, predicted):
        if i == j:
            correct += 1
    return correct/len(row['y_true'])

rules_1['total_accuracy'] = rules_1.apply(total_accuracy, axis=1)

In [14]:
rules_1[['name', 'y_true', 'predicted_label', 'same', 'total_accuracy']].sort_values(['same'])

,name,y_true,predicted_label,same,total_accuracy
14,warrior2_arms_4,"[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",False,0.8
25,warrior2_hips_5,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...",False,0.9
24,warrior2_hips_4,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...",False,0.9
23,warrior2_hips_3,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...",False,0.9
22,warrior2_hips_2,"[1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",False,0.8
21,warrior2_hips_1,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",False,0.8
20,warrior2_torso_5,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...",False,0.9
19,warrior2_torso_4,"[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",False,0.9
17,warrior2_torso_2,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...",False,0.8
16,warrior2_torso_1,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...",False,0.8


In [15]:
np.mean(rules_1['total_accuracy'])

0.9199999999999998

In [16]:
tags_dict

{'arms': 0,
 'front_knee_obtuse': 1,
 'front_knee_acute': 2,
 'head_sideways': 3,
 'hips_angled': 4,
 'narrow_step': 5,
 'shoulders_up': 6,
 'torso_forward': 7,
 'torso_backward': 8,
 'wide_step': 9}

In [17]:
def single_fun_right(row, key):
    # key of pose in key_dict
    index = tags_dict[key]
    true = row['y_true'][index]
    predicted = row['predicted_label'][index]
    if true == predicted:
        return 1.0
    else:
        return 0.0


for key in tags_dict:
    rules_1[key+"_right"] = rules_1.apply(lambda x: single_fun_right(row=x, key=key), axis=1)
    

In [18]:
rules_1.head()

,name,0,1,2,3,4,5,6,7,8,...,arms_right,front_knee_obtuse_right,front_knee_acute_right,head_sideways_right,hips_angled_right,narrow_step_right,shoulders_up_right,torso_forward_right,torso_backward_right,wide_step_right
0,warrior2_knee1_1,704.2094,131.7818,0.873184,652.1342,196.4106,0.881475,591.7704,194.3839,0.768179,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,warrior2_knee1_2,713.5471,175.2373,0.896677,662.6757,239.2642,0.817547,598.0185,229.7448,0.738699,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,warrior2_knee1_3,726.8618,157.8176,0.868085,668.6129,210.1443,0.861277,610.7689,208.9256,0.751413,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
3,warrior2_knee1_4,668.6051,165.1224,0.914809,674.0378,229.7221,0.842912,615.6501,228.5975,0.805993,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
4,warrior2_knee1_5,744.9551,206.4096,0.916538,711.0170,257.0716,0.865699,650.9920,255.7612,0.765953,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0


In [19]:
def total_true_postive(row):
    TP = 0
    true = row['y_true']
    predicted = row['predicted_label']
    for i, j in zip(true, predicted):
        if i == j and j == 1:
            TP += 1
    return TP

rules_1['total_TP'] = rules_1.apply(total_true_postive, axis=1)

In [20]:
def total_false_postive(row):
    FP = 0
    true = row['y_true']
    predicted = row['predicted_label']
    for t, p in zip(true, predicted):
        if t != p and p == 1:
            FP += 1
    return FP

rules_1['total_FP'] = rules_1.apply(total_false_postive, axis=1)

In [21]:
def total_false_negitive(row):
    FN = 0
    true = row['y_true']
    predicted = row['predicted_label']
    for t, p in zip(true, predicted):
        if t != p and p == 0:
            FN += 1
    return FN

rules_1['total_FN'] = rules_1.apply(total_false_negitive, axis=1)

In [22]:
rules_1[['name', 'y_true', 'predicted_label', 'same', 'total_accuracy', 'total_TP', 'total_FP', 'total_FN']].sort_values(['same'])

,name,y_true,predicted_label,same,total_accuracy,total_TP,total_FP,total_FN
14,warrior2_arms_4,"[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",False,0.8,2,0,2
25,warrior2_hips_5,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...",False,0.9,3,0,1
24,warrior2_hips_4,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...",False,0.9,3,0,1
23,warrior2_hips_3,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...",False,0.9,3,0,1
22,warrior2_hips_2,"[1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",False,0.8,3,0,2
21,warrior2_hips_1,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",False,0.8,3,1,1
20,warrior2_torso_5,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...",False,0.9,2,1,0
19,warrior2_torso_4,"[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",False,0.9,3,0,1
17,warrior2_torso_2,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...",False,0.8,2,2,0
16,warrior2_torso_1,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...",False,0.8,2,2,0


In [23]:
def total_precision(row):
    if (row['total_TP']+row['total_FP']) == 0:
        return 0
    else:
        return row['total_TP']/(row['total_TP']+row['total_FP'])

rules_1['total_precision'] = rules_1.apply(total_precision, axis=1)

In [24]:
def total_recall(row):
    if (row['total_TP']+row['total_FN']) == 0:
        return 0
    else:
        return row['total_TP']/(row['total_TP']+row['total_FN'])

rules_1['total_recall'] = rules_1.apply(total_recall, axis=1)

In [25]:
np.mean(rules_1['total_precision'])

0.763888888888889

In [26]:
np.mean(rules_1['total_recall'])

0.77

In [27]:
rules_1[['name', 'y_true', 'predicted_label', 'same', 'total_accuracy', 'total_precision', 'total_recall']].sort_values(['same'])

,name,y_true,predicted_label,same,total_accuracy,total_precision,total_recall
14,warrior2_arms_4,"[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",False,0.8,1.000000,0.50
25,warrior2_hips_5,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...",False,0.9,1.000000,0.75
24,warrior2_hips_4,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...",False,0.9,1.000000,0.75
23,warrior2_hips_3,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...",False,0.9,1.000000,0.75
22,warrior2_hips_2,"[1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",False,0.8,1.000000,0.60
21,warrior2_hips_1,"[1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, ...",False,0.8,0.750000,0.75
20,warrior2_torso_5,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...",False,0.9,0.666667,1.00
19,warrior2_torso_4,"[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",False,0.9,1.000000,0.75
17,warrior2_torso_2,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...",False,0.8,0.500000,1.00
16,warrior2_torso_1,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...",False,0.8,0.500000,1.00


In [43]:
def total_fl(row):
    if (row['total_precision']+row['total_recall']) == 0:
        return None
    return 2*((row['total_precision']*row['total_recall'])/(row['total_precision']+row['total_recall']))

rules_1['total_f1'] = rules_1.apply(total_fl, axis=1)

In [44]:
np.mean(rules_1['total_f1'])

0.8584249084249085

In [31]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

pose_metrics = []

for pose in tags_dict.keys():
    metric = [pose]
    true = rules_1[pose+"_true"]
    predicted = rules_1[pose+"_predicted"]
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(true, predicted)
    precision = precision_score(true, predicted)
    recall = recall_score(true, predicted)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(true, predicted)
    metric.append(accuracy)
    metric.append(precision)
    metric.append(recall)
    metric.append(f1)
    pose_metrics.append(metric)

In [32]:
pose_metrics

[['arms', 0.8666666666666667, 0.9285714285714286, 0.8125, 0.8666666666666666],
 ['front_knee_obtuse', 1.0, 1.0, 1.0, 1.0],
 ['front_knee_acute', 0.9, 0.75, 0.8571428571428571, 0.7999999999999999],
 ['head_sideways', 0.9333333333333333, 0.6666666666666666, 1.0, 0.8],
 ['hips_angled',
  0.8333333333333334,
  0.7,
  0.7777777777777778,
  0.7368421052631577],
 ['narrow_step', 0.9666666666666667, 1.0, 0.8, 0.888888888888889],
 ['shoulders_up',
  0.7666666666666667,
  0.625,
  0.5555555555555556,
  0.5882352941176471],
 ['torso_forward', 0.9333333333333333, 0.75, 0.75, 0.75],
 ['torso_backward', 1.0, 1.0, 1.0, 1.0],
 ['wide_step', 1.0, 1.0, 1.0, 1.0]]

In [33]:
pose_metrics_df = pd.DataFrame(pose_metrics)
pose_metrics_df.columns = ['pose', 'accuracy', 'precision', 'recall', 'f1']
pose_metrics_df

,pose,accuracy,precision,recall,f1
0,arms,0.866667,0.928571,0.812500,0.866667
1,front_knee_obtuse,1.000000,1.000000,1.000000,1.000000
2,front_knee_acute,0.900000,0.750000,0.857143,0.800000
3,head_sideways,0.933333,0.666667,1.000000,0.800000
4,hips_angled,0.833333,0.700000,0.777778,0.736842
5,narrow_step,0.966667,1.000000,0.800000,0.888889
6,shoulders_up,0.766667,0.625000,0.555556,0.588235
7,torso_forward,0.933333,0.750000,0.750000,0.750000
8,torso_backward,1.000000,1.000000,1.000000,1.000000
9,wide_step,1.000000,1.000000,1.000000,1.000000


In [34]:
pose_metrics_df.to_csv("pose_metrics_df_423.csv")

In [35]:
pose_metrics_no_name = []

for pose in tags_dict.keys():
    metric = []
    true = rules_1[pose+"_true"]
    predicted = rules_1[pose+"_predicted"]
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(true, predicted)
    precision = precision_score(true, predicted)
    recall = recall_score(true, predicted)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(true, predicted)
    metric.append(accuracy)
    metric.append(precision)
    metric.append(recall)
    metric.append(f1)
    pose_metrics_no_name.append(metric)

In [37]:
rules_1['y_true']

0     [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1     [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2     [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...
3     [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4     [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...
5     [0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
6     [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
7     [1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
8     [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
9     [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
10    [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
11    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
12    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...
13    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...
14    [1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...
15    [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
16    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...
17    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0

In [38]:
all_true = list(rules_1['y_true'][0])
all_predicted = list(rules_1['predicted_label'][0])

for i in range(1, len(rules_1['y_true'])):
    all_true += list(rules_1['y_true'][i])
    all_predicted += list(rules_1['predicted_label'][i])

In [40]:
all_accuracy = accuracy_score(all_true, all_predicted)
all_precision = precision_score(all_true, all_predicted)
all_recall = recall_score(all_true, all_predicted)
    # f1: 2 tp / (2 tp + fp + fn)
all_f1 = f1_score(all_true, all_predicted)

In [46]:
print("Total accuracy", all_accuracy)
print("Total precision", all_precision)
print("Total recall", all_recall)
print("Total f1", f1)

Total accuracy 0.92
Total precision 0.8181818181818182
Total recall 0.8181818181818182
Total f1 1.0


In [50]:
## Micro Precision
print("Average accuracy", np.mean(rules_1['total_accuracy']))
print("Average precision", np.mean(rules_1['total_precision']))
print("Average recall", np.mean(rules_1['total_recall']))
print("Average f1", np.mean(rules_1['total_f1']))


Average accuracy 0.9199999999999998
Average precision 0.763888888888889
Average recall 0.77
Average f1 0.8584249084249085
